# 文章データへの前処理
今回のサンプルでは、下記のように文書への前処理を行います。<br>

- 後の扱いを考え、文字コードをUTF-8に統一（`Webスクレイピング時点で実施済み`）
- Pythonライブラリ BeautifulSoup を使用しhtmlタグを除去（`Webスクレイピング時点で実施済み`）
- 表記のゆらぎの修正　→　Pythonライブラリ neologdn を使用
- 形態素解析ソフト MeCab を使用し、単語分割し不要ワードを除去する


In [ ]:
## 必要なライブラリの読み込み
import sys
import re
import random
import glob
## 【注】以下の2つを使用するには事前準備が必要（別途資料参照）
import MeCab
import neologdn

### preprocess_text_1
与えられた文書の文字列（input_text）を前処理・単語分割し、必要な単語の羅列（List型）を返す関数を定義する。<br>
※ この処理はあくまで１例

In [ ]:
def preprocess_text_1(input_text):
    input_text_reg = neologdn.normalize(input_text)     ### neologdn による表記ゆれ正規化 -> input_text_reg
    tagger = MeCab.Tagger("-Ochasen")                   ### MeCab による日本語形態素解析
    node = tagger.parseToNode(input_text_reg)           ###    -> input_text_reg を解析した結果がnodeに入る
    res = []                                            ### 結果の戻り値用入れ物
    while node is not None:
        ### nodeの中にはfeatureプロパティ、surfaceプロパティがある
        sur = node.surface   ### node.surface は 分割した単語そのもの
        ### node.feature はsurfaceに対応するcsv文字列: 品詞,品詞細分類1,分類2,分類3,活用形,活用型,原形,読み,発音
        a = node.feature.split(',')
        hin = a[0]           ### 品詞
        bun1 = a[1]          ### 品詞細分類1
        if hin == '動詞':    ### 動詞は原形に直しておく
            sur = a[6]
        ### 品詞、細分類1 で不要そうなカテゴリのもの、数字や記号カッコ入り、長さが１、の場合を除外し、それ以外の単語を拾う
        if not( ( hin in ['BOS/EOS','連体詞','接続詞','助詞','助動詞','連語','副詞','接頭詞','記号'] ) or \
           ( bun1 in ['形容動詞語幹','副詞可能','代名詞','ナイ形容詞語幹','特殊','数','接尾','非自立'] ) or \
           ( hin == '名詞' and ( re.search('\d',sur) or re.search('[\(\)\（）]',sur) ) ) or len(sur) <= 1 ) :
            res.append(sur)
        node = node.next
    return res

### preprocess_file_1
与えられたファイル（複数）の中身の文書テキストを preprocess_text_1 で前処理し、まとめてリストとして返す関数を定義

In [ ]:
def preprocess_file_1(input_files):
    res = []
    for input_file in input_files:
        with open(input_file, 'r', encoding='utf-8') as input_file_hd:
            res += preprocess_text_1( input_file_hd.read() )
    return res

Webスクレイピングで取得したファイルを前処理して、１行１単語の前処理済みテキストとして書き出す。<br>
- mlit_y27.txt , mlit_y28.txt , mlit_y29.txt の３つを前処理 → mlit.txt へ
- env_y27.txt , env_y28.txt , env_y29.txt の３つを前処理 → env.txt へ
ここで作った mlit.txt と env.txt は学習の為のデータとして使います。<br>
glob.globを使うと、存在するファイルのリストを作るのに便利。又、List型を文字列として連結する際はjoinを使うのが便利。

In [ ]:
with open('mlit.txt', 'w', encoding='utf-8') as mlit_out_file:
    mlit_out_file.write( "\n".join( preprocess_file_1(glob.glob('mlit_y2[789].txt')) ) + "\n" )

with open('env.txt', 'w', encoding='utf-8') as env_out_file:
    env_out_file.write( "\n".join( preprocess_file_1(glob.glob('env_y2[789].txt')) ) + "\n" )

[glob.glob('mlit.txt'),glob.glob('env.txt')]    # ファイルが出来たか確認用

検証用データとして使う為、mlit,envの両方で、30年度のテキストの中ランダムな場所から、全体の1%を抜き出しておく。

In [ ]:
test_ratio = 0.01

with open('mlit_check.txt', 'w', encoding='utf-8') as mlit_out_file:
    a = preprocess_file_1(['mlit_y30.txt'])
    r = random.random()
    mlit_out_file.write( ("\n".join(a[int(r * (1.0 - test_ratio)):][0:int(test_ratio * len(a))])) + "\n" )

with open('env_check.txt', 'w', encoding='utf-8') as env_out_file:
    a = preprocess_file_1(['env_y30.txt'])
    r = random.random()
    env_out_file.write( ("\n".join(a[int(r * (1.0 - test_ratio)):][0:int(test_ratio * len(a))])) + "\n" )

[glob.glob('mlit_check.txt'),glob.glob('env_check.txt')]    # ファイルが出来たか確認用